In [50]:
#Split in Train and Test
from time import time
import logging

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sklearn.model_selection import train_test_split

In [51]:
spark = SparkSession.builder.master("spark://spark-master-01:7077").enableHiveSupport().getOrCreate()

In [52]:
spark.sql("show tables").show()

+--------------------+-----------+
|           tableName|isTemporary|
+--------------------+-----------+
|carshare_historic...|      false|
|     carshare_sample|      false|
| carshare_sample_1mm|      false|
|       customer_data|      false|
|          customerdm|      false|
|       customer_eval|      false|
|    customer_grouped|      false|
|   customer_training|      false|
|demo3_train_obd_data|      false|
|demo3_train_obd_test|      false|
|          got_tweets|      false|
|             got_web|      false|
|          prediction|      false|
|             titanic|      false|
+--------------------+-----------+



In [53]:
df_train =  spark.sql("""SELECT ROW_NUM, TrialID, ObsNum, CAST(IsAlert AS DOUBLE) AS label, P1, P2, P3 , P4, P5,\
    P6 , P7, P8 ,  E1, E2, E3 ,E4 ,\
    E5, E6 , E7 , E8 , E9 , E10 ,\
    E11 , V1, V2, V3 , V4, V5 , V6 , V7 ,\
    V8, V9 , V10 , V11 \
    FROM demo3_train_obd_data""").na.drop()


In [54]:
pd_train = df_train.toPandas()

In [55]:
prop = df_train.groupBy('label').count().toPandas()

In [56]:
#print prop[prop['label']==1]['count']v
baseline = prop[prop['label']==1]['count'].sum() / (1.0*(prop[prop['label']==1]['count'].sum()+prop[prop['label']==0]['count'].sum()))
print baseline

0.578798965464


In [57]:
y = pd_train['label']
X = pd_train.drop('label', 1)

In [58]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state=0)

In [81]:
data_train = X_train
data_train['label'] = pd.Series(y_train, index=X_train.index)

data_test = X_test
data_test['label'] = pd.Series(y_test, index=X_test.index)


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [82]:
print data_test

        ROW_NUM  TrialID  ObsNum        P1        P2    P3        P4  \
519169   519169      429     591   31.9833  10.96020   852   70.4225   
373953   373953      309     453   29.0343  13.79750  1948   30.8008   
275732   275732      228     176   30.8788   9.29190  1200   50.0000   
338744   338744      280     335   32.5191  13.36230  1728   34.7222   
181469   181469      150     173   37.5742   8.20535   992   60.4839   
472008   472008      390     565   35.1044  11.65560  1424   42.1348   
318871   318871      263    1032   38.0095  13.70210  1136   52.8169   
42771     42771       35     460   31.2091  11.32220  1292   46.4396   
431770   431770      357     169   31.3378   6.69115  1264   47.4684   
111212   111212       92       8  100.4480   9.62620   960   62.5000   
576025   576025      487     654   34.1449  18.49510   980   61.2245   
318757   318757      263     918   28.3054  10.22660  1048   57.2519   
420799   420799      348      97   23.3458   9.96679   592  101.

In [83]:
pd_train.loc[519169]

ROW_NUM    519169.000000
TrialID       429.000000
ObsNum        591.000000
label           1.000000
P1             31.983300
P2             10.960200
P3            852.000000
P4             70.422500
P5              0.097299
P6            844.000000
P7             71.090000
P8              0.000000
E1              0.018000
E2            164.282000
E3              0.000000
E4              0.000000
E5              0.016001
E6            367.000000
E7              0.000000
E8              0.000000
E9              1.000000
E10            41.000000
E11            23.600000
V1              0.000000
V2              0.175000
V3            496.000000
V4             34.518700
V5              1.000000
V6            631.000000
V7              0.000000
V8              0.000000
V9              0.000000
V10             1.000000
V11            16.610200
Name: 519169, dtype: float64

In [84]:
data_test.loc[519169]

ROW_NUM    519169.000000
TrialID       429.000000
ObsNum        591.000000
P1             31.983300
P2             10.960200
P3            852.000000
P4             70.422500
P5              0.097299
P6            844.000000
P7             71.090000
P8              0.000000
E1              0.018000
E2            164.282000
E3              0.000000
E4              0.000000
E5              0.016001
E6            367.000000
E7              0.000000
E8              0.000000
E9              1.000000
E10            41.000000
E11            23.600000
V1              0.000000
V2              0.175000
V3            496.000000
V4             34.518700
V5              1.000000
V6            631.000000
V7              0.000000
V8              0.000000
V9              0.000000
V10             1.000000
V11            16.610200
label           1.000000
Name: 519169, dtype: float64

In [87]:
data_train.drop('ROW_NUM', 1).to_csv('obd_data_for_train.csv')
data_test.drop('ROW_NUM', 1).to_csv('obd_data_for_test.csv')